In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import random
import math


''' WHAT ALL OUR INPUTS ARE REFERRING TO
Ed = energy depositted in scatterer
Ea = energy in absorber
angle_s = scattering angle (found from energy depositted in scatterer)
angle_a = scattering angle (found from energy depositted in absorber)
err_s = error on scattering angle (found from energy depositted in scatterer)
err_a = error on scattering angle (found from energy depositted in absorber)
ss = absolute distance from source to scatterer
sa = absolute distance from scatterer and absorber
res_s = resolution of scatterer
res_a = resolution of absorber
'''

'''When a depositted energy (in scatterer) is known, output the scattering angle and its associated error due to energy res'''
'''If we have a compton scatter in the absorber before the photopeak - we cant use the information that absorber gives us, and so only scatter angle and error is used'''
def s_angle_s(Ed, res): 
    Eg = 1.06*10**(-13) # This is the initial gamma ray energy from the source in keV
    m = 9.11 * 10**(-31) # mass of electron
    c = 3 * 10**8 # normal c
    alpha = (m*(c**2))
    constant = res * Eg**(0.5) #res given from manual for now
    scat_a = math.acos(1 - ((m*c**2)*((1/(Eg-Ed)) - (1/Eg)))) #compton formula
    res1 = constant / (Ed)**0.5 #resolution based on the actual energy depositted and calculated constant
    Eerr1 = Ed * res1 #energy error
    scat_a_err = Eerr1 * (((Eg*(1 - math.cos(scat_a)) + alpha)**2)/((Eg**2)*alpha*math.sin(scat_a))) #error on scattering angle
    #from error prop of compton (based off readings from scattering detector)
    
    return scat_a, scat_a_err

'''When a depositted energy (in absorber) is known, output the scattering angle and its associated error due to energy res'''
def s_angle_a(Ea, res):
    Eg = 1.06*10**(-13) # This is the initial gamma ray energy from the source in keV
    m = 9.11 * 10**(-31) # mass of electron
    c = 3 * 10**8 # normal c
    alpha = m*(c**2)
    constant = res * Eg**(0.5)
    scat_a = math.acos(1 - ((m*c**2)*((1/Ea) - (1/Eg))))
    res1 = constant / (Ea)**0.5 
    Eerr1 = Ea * res1
    scat_a_err = Eerr1 * (((Eg*(1 - math.cos(scat_a)) + alpha)**2)/((Eg**2)*alpha*math.sin(scat_a)))
    
    return scat_a, scat_a_err

'''Outputs weighted mean and associated error on mean for angle due to res of both detectors - when we have our ideal reactions occuring'''
'''only used when we can use both energies from scatterer and absorber - if not we will be able to use the energy from scatter (to calc angle)'''
def w_mean(angle_s, err_s, angle_a, err_a):
    w_mean = ((angle_s/(err_s)**2) + (angle_a/(err_a)**2)) / ((err_s)**-2 + (err_a)**-2)
    w_mean_err = 1/ ((err_s)**-2 + (err_a)**-2)
    return w_mean, w_mean_err


'''Error contribution due to geometry of set up'''
def geo_err(ss, sa): #absolute distances in cm
    err_ss = math.atan(2.54/(2*ss)) #assuming it could hit anywhere in detector 
    er_sa = math.atan((4/3)*(2.225523/sa)) #assuming it only hits middle of detector (within 2.225523 in centre, from J & J gaussian data curve for sigma)
    tot_err = (err_ss**2 + er_sa**2)**0.5 #contribution from both ss and sa
    #^using weighted mean decreases error, so ideally use this however due to the fact the absorber may compton scatter before full
    #absorbtion we may have to only use the error from the scatterer (increased error)
    return tot_err


'''function that works out the total angular resolution from the following known parameters 
under assumption we have 1 scatter and 1 full absorbtion'''
def tot_err_ideal(ss, sa, Ea, res_a, Ed, res_s):
    angle_a = s_angle_a(Ea, res_a)[0] #gives scattering angle from absorber
    angle_s = s_angle_s(Ed, res_s)[0] #gives scattering angle from scatterer
    err_a = s_angle_a(Ea, res_a)[1] #error on angle from absorber (based on res)
    err_s = s_angle_s(Ed, res_s)[1] #error on angle from scatterer (based on res)
    E_angle = ((angle_s/(err_s)**2) + (angle_a/(err_a)**2)) / ((err_s)**-2 + (err_a)**-2) #weighted mean for angle
    E_err = 1 / ((err_s)**-2 + (err_a)**-2) #weighted mean for angle error (due to res)
    err_ss = math.atan(2.54/(2*ss)) #geometry cont.
    err_sa = math.atan((4/3)*(2.225523/sa)) #geometry cont.     
    geo_err = (err_ss**2 + err_sa**2)**0.5
    tot_err = (geo_err**2 + E_err**2)**0.5 #outputs total error (for the ideal interaction process)
    return tot_err

'''total error, only using information from the scatterer'''
def tot_err_s(ss, sa, Ed, res_s):
    angle_s = s_angle_s(Ed, res_s)[0] #gives scattering angle from scatterer
    err_s = s_angle_s(Ed, res_s)[1] #error on angle from scatterer (based on res)
    err_ss = math.atan(2.54/(2*ss)) #geometry cont.
    err_sa = math.atan((4/3)*(2.225523/sa)) #geometry cont.     
    geo_err = (err_ss**2 + err_sa**2)**0.5
    tot_err = (geo_err**2 + err_s**2)**0.5 #outputs total error (for the ideal interaction process)
    return tot_err


print((tot_err_ideal(20, 80, 8.992*10**-14, 0.08, 1.6*10**-14, 0.075))*(360/(2*math.pi)))
print((tot_err_s(20, 80, 1.6*10**-14, 0.075))*(360/(2*math.pi)))

4.213048259283182
5.499373044321052
